# Customer Churn Prediction using Deep Learning

In [1]:
import pandas as pd
import numpy as np
import keras 
import matplotlib.pyplot as plt

%matplotlib inline

C:\Users\amitr\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Read in the data

In [3]:
# Read in the data
df = pd.read_csv(r'C:\Users\amitr\OneDrive\Desktop\Deep Learning\Data\Churn_Modelling.csv')
df.drop(['RowNumber','CustomerId','Surname'], axis=1, inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### Data Preprocessing

In [20]:
# Create Training and Test sets

from sklearn.model_selection import train_test_split

y = df['Exited']
X = df.drop('Exited',axis=1)

X = pd.get_dummies(X,drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


#### Feature Scaling

In [21]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Create the Artificial Neural Network

In [24]:
import keras
from keras.models import Sequential
from keras.layers import Dense

##### How to choose the nodes in the hidden layer

It's an art and there is no specific rule. However, a good starting point is to select an average of input and output nodes.

In this case we have 11 input variables and 1 output (yes/no)...hence we can consider (11 + 1)/ 2 = 6 nodes

##### Parameters to specify for the hidden layers we add
1) Units --> Dimensionality of the output space. Same as output_dim in the earlier versions. Set his to 6 as above

2) Kernel_initializer --> use 'uniform'. That assigns a small starting weight to each node considering a normal distribution

3) Activation --> We'll use the ReLU (Rectified Linear Unit) function for the hidden layers, plus a sigmoid for the final output layer

4) input_dim --> this is also mandatory. Set it as the number of independent variables in the input data (use df.shape to know the number of inputs)

In [40]:
# Create the layers
classifier = Sequential()

# Adding the first hidden layer
classifier.add(Dense(units=6,
                     kernel_initializer='uniform',
                     activation='relu',
                     input_dim=11))

# Adding the second hidden layer
classifier.add(Dense(units=6,
                     kernel_initializer='uniform',
                     activation='relu'))  # don't need to specify the input_dim for subsequent layers

# Add the final output layer
classifier.add(Dense(units=1,   # we have a binary outcome Yes/No
                     kernel_initializer = 'uniform',
                     activation = 'sigmoid'))  # for more than 2 categories use 'softmax'

##### Compiling the network

This adds gradient descent to the Neural Network and makes it ready to execute

1) optimizer --> We have various gradient descent algorithms like stochasticetc. We'll use a common one called 'adam'

2) loss function --> since we are using the sigmoid function in the last layer this should be a logarithmic function (same as used for logistic regression). Set it as binary_crossentropy

3) metrics -> method to evaluate the model after every batch to improve the model. 'accuracy' is the most common

In [41]:
# Compile the ANN
classifier.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

##### Now we have configured the ANN. Next we'll fit it to our data

1) batch_size --> Specify whether the weights would be updated after each observation (Reinforcement learning) or after several observations (batch learning)

2) epochs --> how many times to loop through the entire data

In [42]:
# Fit the ANN to the data
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 3s 373us/step - loss: 0.4854 - acc: 0.7969
Epoch 2/100
8000/8000 [==============================] - 1s 128us/step - loss: 0.4225 - acc: 0.8186
Epoch 3/100
8000/8000 [==============================] - 1s 150us/step - loss: 0.4152 - acc: 0.8277
Epoch 4/100
8000/8000 [==============================] - 1s 148us/step - loss: 0.4101 - acc: 0.8305
Epoch 5/100
8000/8000 [==============================] - 1s 131us/step - loss: 0.4060 - acc: 0.8342
Epoch 6/100
8000/8000 [==============================] - 1s 130us/step - loss: 0.4034 - acc: 0.8339
Epoch 7/100
8000/8000 [==============================] - 1s 136us/step - loss: 0.4018 - acc: 0.8367
Epoch 8/100
8000/8000 [==============================] - 1s 141us/step - loss: 0.4007 - acc: 0.8369
Epoch 9/100
8000/8000 [==============================] - 1s 142us/step - loss: 0.3997 - acc: 0.8359
Epoch 10/100
8000/8000 [==============================] - 1s 130us/step - loss: 0.3988 - acc: 0.8377

##### Now time to predict the outcomes

This is the same as classical models. We'll follow the same steps

In [52]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_pred = classifier.predict(X_test)
y_pred = y_pred > 0.5

print ("Accuracy on test set : ", accuracy_score(y_test, y_pred))

print ("\nConfusion Matrix : \n", confusion_matrix(y_test, y_pred))

Accuracy on test set :  0.8585

Confusion Matrix : 
 [[1528   58]
 [ 225  189]]


#### Homework assignment : Predicting for a specific customer

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: $50000

So should we say goodbye to that customer ?

In [74]:
# Select one row and edit it

X_cust = X[:1].copy()

X_cust['CreditScore'] = 600
X_cust['Age'] = 40
X_cust['Tenure'] = 3
X_cust['Balance'] = 60000
X_cust['NumOfProducts'] = 2
X_cust['EstimatedSalary'] = 50000
X_cust['Gender_Male'] = 1

X_cust

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,600,40,3,60000,2,1,1,50000,0,0,1


In [75]:
# Scale the data
X_cust = sc.transform(X_cust)
X_cust

array([[-0.51795799,  0.10698342, -0.69673114, -0.26352876,  0.79286681,
         0.64298333,  0.97067965, -0.86629511, -0.57946723, -0.57677292,
         0.90682052]])

In [84]:
# Predict if we should retain the customer

pred = classifier.predict(X_cust)

decision = np.where (pred > 0.5, "Yes","No")

print ("Should we retain the customer? : ", decision)

Should we retain the customer? :  [['No']]
